In [8]:
import json
import re
import copy

#  用来判断所有纯编码实体(如：1.2.3.1、1.2.3.-)是否均来源于 uniprot实体库，是的。   
#with open('./medical_entity_dict_v3.0.txt','r') as file, open('./uniprot_number_entity.txt','w') as fl:
with open('./medical_entity_dict_v3.0.txt','r') as file, open('./medical_entity_dict_v4.0.txt','w') as fl:
#    pattern = re.compile(r"\>+\d+\.\-?\d{0,10}\.\-?\d{0,10}\.\-?\d{0,10}\<")
    pattern = re.compile(r'"\d+\.\-?\d{0,10}\.\-?\d{0,10}\.\-?\d{0,10}"')
    name_count = 0
    synonym_count = 0
    
    file.seek(0,0)
    for line in file:
        line_dict = json.loads(line.strip())
#        line_result = pattern.findall(line)
#        if line_result != []:
#            fl.write(json.dumps(line_dict) + "\n")
        
        name = line_dict["name"]
        synonym_list = line_dict["synonym_list"]
        source = line_dict["source"]
        
        name_result = pattern.findall('"{}"'.format(name))
        if name_result != []:
            name_count += 1
            line_dict["name"] = "ec " + name
#             fl.write(json.dumps(line_dict) + "\n")
            
        new_synonym_list = copy.copy(synonym_list)
        for synonym in synonym_list:
            synonym_result = pattern.findall('"{}"'.format(synonym))
            if synonym_result != []:
                synonym_count += 1
                
                new_synonym_list.remove(synonym)
#                 new_synonym = "EC " + synonym
                new_synonym = "ec {}".format(synonym)
                new_synonym_list.append(new_synonym)
        line_dict["synonym_list"] = new_synonym_list
        fl.write(json.dumps(line_dict) + "\n")
        
    print(name_count)
    print(synonym_count)
    

0
5007


In [9]:
#  用于将实体字典中所有纯编码实体 都改为以“EC”开头的实体
import json
import re
import copy
import pickle

with open('./medical_entity_dict_v4.5.pkl','rb') as file, open('./medical_entity_dict_v4.6.pkl','wb') as fs:
    med_synonyms_name_dict, med_name_cate_dict = pickle.loads(file.read())
    new_med_synonyms_name_dict = {}
    new_med_name_cate_dict = copy.copy(med_name_cate_dict)
    pattern = re.compile(r'"\d+\.\-?\d{0,10}\.\-?\d{0,10}\.\-?\d{0,10}"')
    k_count = 0
    v_count = 0
    key_count = 0
    for k,v in med_synonyms_name_dict.items():
        k_result = pattern.findall('"{}"'.format(k))
        v_result = pattern.findall('"{}"'.format(v))
        if k_result != []:
            k_count += 1
            print(k)
            new_k = "ec {}".format(k)
        else:
            new_k = k
            
        if v_result != []:
            v_count += 1
            new_v = "ec {}".format(v)
        else:
            new_v = v
        new_med_synonyms_name_dict[new_k] = new_v
        
    for key,value in med_name_cate_dict.items():
        key_result = pattern.findall('"{}"'.format(key))
#         value_result = pattern.findall('"{}"'.format(value))
        if key_result != []:
            key_count += 1
            new_key = "ec {}".format(key)
            new_med_name_cate_dict.pop(key)
            new_med_name_cate_dict[new_key] = value
    
    dict_concat = [new_med_synonyms_name_dict, new_med_name_cate_dict]
    pickle.dump(dict_concat, fs)
    print(k_count)
    print(v_count)
    print(key_count)
    
    

1.8.1.-
3.5.-.-
1.8.3.-
4.3.-.-
2.3.-.-
1.18.1.-
2.6.1.-
1.3.3.-
4.2.3.-
1.8.5.-
3.1.5.-
1.2.4.-
1.6.5.-
3.1.11.-
3.3.-.-
3.2.-.-
6.3.4.-
2.1.-.-
1.1.99.-
2.3.3.-
1.13.-.-
4.3.2.-
3.3.2.-
2.2.1.-
2.6.99.-
1.17.3.-
2.8.1.-
1.8.-.-
4.1.-.-
1.14.18.-
30
0
0


In [10]:
#  用于将关系字典中所有纯编码实体 都改为以“EC”开头的实体
import json
import os
import re
import copy

file_folder_path = r"/home/chihui/chihui/jupyter/data/relationship_dict/all_relation_dict_file_v2.0/quchong_relation_dict_v2.1"
file_path_list = []
for file_path_root,_,file_name_list in os.walk(file_folder_path):
    for file_name in file_name_list:
        if ".dict" in file_name:
            file_path_list.append(os.path.join(file_path_root,file_name))
            
num = 0
for file_name in file_path_list:
    num = num + 1
    print(str(num) + ":" + os.path.basename(file_name))
#  将关系字典每个文件中的纯编码实体 6.3.4.- 都纠正为：EC 6.3.4.-  
    with open(file_name,'r') as file, open('/home/chihui/chihui/jupyter/data/relationship_dict/all_relation_dict_file_v2.0/quchong_relation_dict_v2.2/' + os.path.basename(file_name), 'a') as fl:
        pattern = re.compile(r'"\d+\.\-?\d{0,10}\.\-?\d{0,10}\.\-?\d{0,10}"')
        e1_number_count = 0
        e2_number_count = 0
        
        file.seek(0,0)
        for line in file:
            line_dict = json.loads(line.strip())
            e1 = line_dict["e1"]
            e2 = line_dict["e2"]
            sources = line_dict["sources"]
            if "-6" in sources:
                e1_result = pattern.findall('"{}"'.format(e1))
                e2_result = pattern.findall('"{}"'.format(e2))
                if e1_result != []:
                    e1_number_count += 1
                    new_e1 = "ec {}".format(e1)
                    line_dict["e1"] = new_e1
                if e2_result != []:
                    e2_number_count += 1
                    new_e2 = "ec {}".format(e2)
                    line_dict["e2"] = new_e2
                    
            fl.write(json.dumps(line_dict) + "\n")
        print("e1_number_count:" + str(e1_number_count))
        print("e2_number_count:" + str(e2_number_count))
        
        
        

1:compound_protein_treat.dict
e1_number_count:0
e2_number_count:1
2:gene_organism_belong.dict
e1_number_count:0
e2_number_count:0
3:equipment_disease_diagnose|treat.dict
e1_number_count:0
e2_number_count:0
4:drugs_anatomy_affect.dict
e1_number_count:0
e2_number_count:0
5:compound_disease_cause.dict
e1_number_count:0
e2_number_count:0
6:gene_disease_cause.dict
e1_number_count:0
e2_number_count:0
7:gene_protein_express.dict
e1_number_count:0
e2_number_count:3342
8:disease_anatomy_occur.dict
e1_number_count:0
e2_number_count:0
9:compound_drugs_contain.dict
e1_number_count:0
e2_number_count:0
10:protein_anatomy_belong.dict
e1_number_count:0
e2_number_count:0
11:gene_gene_interact.dict
e1_number_count:0
e2_number_count:0
12:gene_anatomy_locate.dict
e1_number_count:0
e2_number_count:0
13:drugs_disease_treat.dict
e1_number_count:0
e2_number_count:0
14:disease_disease_coexist.dict
e1_number_count:0
e2_number_count:0
15:drugs_protein_target.dict
e1_number_count:0
e2_number_count:0
16:protein_pr